<a href="https://colab.research.google.com/github/kyzira/Train-and-Run-LLama3.1-on-Colab/blob/main/Load_and_Use_your_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the second part to your trained model

If you have already created the gguf and the Modelfile you can skip these steps!

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [2]:
import shutil
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from unsloth import FastLanguageModel
import os

# 1. Definiere die Pfade
# ERSETZE DIES DURCH DEN PFAD ZU DEINEM GESPEICHERTEN LORA-ORDNER
lora_model_path = "/content/drive/My Drive/lora_model"

gguf_output_directory = "/content/drive/My Drive/my_trained_llama"
os.makedirs(gguf_output_directory, exist_ok=True)


# 2. Lade das Modell und die LoRA-Gewichte
# Beim Laden für die GGUF-Konvertierung MUSST du load_in_4bit auf False setzen.
# model_name sollte dein LORA-Ordnerpfad sein. Unsloth lädt automatisch das Basismodell und mergt die LoRA-Gewichte.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = lora_model_path, # Pfad zu deinen LoRA-Adaptern
    max_seq_length = 8192,         # Verwende die max_seq_length, die du beim Training verwendet hast
    dtype = None,                  # Unsloth wählt automatisch die beste dtype
    load_in_4bit = False,          # Wichtig: Muss False sein für die Konvertierung
)

# 3. Konvertierung nach GGUF
# 'q4_k_m' ist empfohlen für Ollama (gute Balance zwischen Größe und Qualität)
model.save_pretrained_gguf(
    gguf_output_directory,
    tokenizer,
    quantization_method = "q4_k_m"
)

print(f"\n✅ GGUF-Konvertierung abgeschlossen! Die Datei(en) befinden sich im Ordner: {gguf_output_directory}")
print(f"Die GGUF-Datei hat den Namen: {gguf_output_directory}/model-unsloth-Q4_K_M.gguf")

NotImplementedError: Unsloth cannot find any torch accelerators? You need a GPU.

In [ ]:
# Modelfile erstellen
import os

modelfile_content = """# Modelfile
FROM /content/drive/My Drive/my_trained_llama/model-unsloth-Q4_K_M.gguf
TEMPLATE \"\"\"{% if system %}<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{{ system }}<|eot_id|>{% endif %}<|start_header_id|>user<|end_header_id|>

{{ prompt }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

\"\"\"
STOP "<|eot_id|>"
STOP "<|end_of_text|>"
"""

with open(os.path.join(gguf_output_directory, "Modelfile"), "w") as f:
    f.write(modelfile_content)

print("✅ Modelfile erstellt.")

# If you already have Created a gguf and Modelfile continue from here

In [ ]:
# Ollama installieren
!curl -fsSL https://ollama.com/install.sh | sh

# Ollama-Dienst im Hintergrund starten
import subprocess
import time
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)
print("\n✅ Ollama Service gestartet.")

In [ ]:
import shutil
import os

from google.colab import drive
drive.mount('/content/drive')

# Importiere das Modell in Ollama.
# "mein-llama-31" ist der Name, den du deinem Modell gibst.
!ollama create mein-llama-31 -f "/content/drive/My Drive/my_trained_llama/Modelfile"

print("\n🎉 Modell erfolgreich in Ollama importiert!")

In [ ]:
!ollama run mein-llama-31 "Was ist das Wichtigste, das ich über das Finetuning mit LoRA wissen sollte?"